In [6]:
#Import the necessary packages

from astropy.table import Table
from matplotlib import pyplot as plt
%matplotlib inline
import os
import numpy as np
from astropy.io.ascii import masked
from astropy.io import ascii
import glob
from astropy.io import fits
import wget
import matplotlib.image as mpimg
from astropy.wcs import WCS
from scipy.stats import scoreatpercentile
from astropy.visualization import simple_norm
from reproject import reproject_interp
import sys
from IPython.display import clear_output
from photutils.detection import DAOStarFinder
from astropy.stats import sigma_clipped_stats
from photutils.aperture import CircularAperture
from astropy.visualization import SqrtStretch
from astropy.visualization import ImageNormalize
from astropy.visualization import LogStretch
from astropy.wcs import WCS
import glob
import os
from scipy.stats import scoreatpercentile
import astropy.units as u
from IPython.display import clear_output
from matplotlib import colors
import warnings
import csv
warnings.filterwarnings('ignore')

mycolors = plt.rcParams['axes.prop_cycle'].by_key()['color']

In [7]:
#Set the home path

os.environ['HOME'] ='C:/Users/USER/Documents/GitHub'
homedir = os.getenv("HOME")
tabledir = homedir+'/Virgo/tables'
plotdir = homedir+'/Virgo/plots'
datadir = homedir+'/HTML-building/galaxy'

In [8]:
def imdisplay(image,x, y, width=100, height=100, v1perc=10, v2perc=95, logscale=True):
    '''
    display an image 
    OPTIONAL KEYWORD PARAMETERS
    v1perc: one end of the colormap assigned to the v1perc percent lowest flux 
    v2perc: the other end of the colormap assigned to the v2perc percent highest flux    
    '''
    # make sure image is an np array
    nimage = np.array(image)
    # determine the pixel values at the v1perc and v2perc percentiles
    v1 = scoreatpercentile(nimage, v1perc)
    v2 = scoreatpercentile(nimage, v2perc)
    norm = None
    if logscale:
        norm = ImageNormalize(vmin=v1, vmax=v2, stretch=LogStretch())
    else:
        norm = ImageNormalize(vmin=v1, vmax=v2)
    # Plot the full image
    fig, ax = plt.subplots(figsize=(10, 10))  # You can adjust the size of the figure here
    im = ax.imshow(nimage, origin='lower', norm=norm)
    # Adjust the view to focus on (x, y) without cropping the image
    x_min = max(x - width // 2, 0)
    x_max = min(x + width // 2, nimage.shape[1])
    y_min = max(y - height // 2, 0)
    y_max = min(y + height // 2, nimage.shape[0])
    # Set the axis limits so that (x, y) is at the center
    ax.set_xlim(x - width // 2, x + width // 2)
    ax.set_ylim(y - height // 2, y + height // 2)
    # Hide axis ticks and labels
    plt.axis('off')   

    return fig,ax
    #fig.colorbar(fraction=.08)
def find_files(destination_folder, partial_name):
    matching_files = []

    for root, dirs, files in os.walk(destination_folder):
        for file in files:
            if partial_name.lower() in file.lower():
                matching_files.append(os.path.join(root, file))

    return matching_files 
# Function to convert RA/DEC to pixel coordinates
def convert_radec_to_pixel(ra, dec, header):
    wcs = WCS(header)
    x, y = wcs.all_world2pix(ra, dec, 0)  # Convert RA/DEC to pixel coordinates
    return x, y

In [11]:
# Initialize list to store data for CSV export
output_data = []
csv_file = tabledir + '/Photometrytesting.csv'
galaxy = Table.read(csv_file)

# Add header for the output CSV
output_data.append(['VFID', 'galaxy_name', 'v1perc_blue', 'v2perc_blue', 'v1perc_green', 'v2perc_green', 'v1perc_red', 'v2perc_red'])

for i in range(len(galaxy)):
    galaxy_name = str(galaxy['GALAXY'][i])
    path = datadir + '/pipeline/' + galaxy_name
    VFID = f"VFID{int(galaxy['VF_ID'][i]):04d}"
    ra = galaxy['RA_MOMENT'][i]  # Reading RA
    dec = galaxy['DEC_MOMENT'][i]  # Reading DEC
    v1perc_blue = v2perc_blue = v1perc_green = v2perc_green = v1perc_red = v2perc_red = None  # Initialize percentile values

    # Blue band processing
    if os.path.exists(path):
        destination_folder = path + '\\HPPUNIMAPB'
        partial_name = 'hpacs_25HPPUNIMAPB'
        found_files = find_files(destination_folder, partial_name)
        if found_files:
            found_file = found_files[0]
            data, header = fits.getdata(found_file, header=True)
            x, y = convert_radec_to_pixel(ra, dec, header)
            nimage = np.array(data)

            for v1perc in np.arange(0.1, 40.0, 0.1):
                v1 = scoreatpercentile(nimage, v1perc)
                if v1 > 0:
                    break
            for v2perc in np.arange(99.9, 20.0, -0.1):
                v2 = scoreatpercentile(nimage, v2perc)
                if not np.isnan(v2):
                    break
            v1perc_blue = v1perc
            v2perc_blue = v2perc

            imdisplay(data, x, y, 350, 350, v1perc, v2perc, logscale=True)
            plt.subplots_adjust(wspace=0, hspace=0)
            plt.savefig(datadir + '/png/' + VFID + '-' + galaxy_name + 'blue.png', dpi=150)
            plt.close()

    # Green band processing
    destination_folder = path + '\\HPPUNIMAPG'
    partial_name = 'hpacs_25HPPUNIMAPB'
    found_files = find_files(destination_folder, partial_name)
    if found_files:
        found_file = found_files[0]
        data, header = fits.getdata(found_file, header=True)
        x, y = convert_radec_to_pixel(ra, dec, header)
        nimage = np.array(data)

        for v1perc in np.arange(0.1, 40.0, 0.1):
            v1 = scoreatpercentile(nimage, v1perc)
            if v1 > 0:
                break
        for v2perc in np.arange(99.9, 20.0, -0.1):
            v2 = scoreatpercentile(nimage, v2perc)
            if not np.isnan(v2):
                break
        v1perc_green = v1perc
        v2perc_green = v2perc

        imdisplay(data, x, y, 350, 350, v1perc, v2perc, logscale=True)
        plt.subplots_adjust(wspace=0, hspace=0)
        plt.savefig(datadir + '/png/' + VFID + '-' + galaxy_name + 'green.png', dpi=150)
        plt.close()

    # Red band processing
    destination_folder = path + '\\HPPUNIMAPR'
    partial_name = 'hpacs_25HPPUNIMAPR'
    found_files = find_files(destination_folder, partial_name)
    if found_files:
        found_file = found_files[0]
        data, header = fits.getdata(found_file, header=True)
        x, y = convert_radec_to_pixel(ra, dec, header)
        nimage = np.array(data)

        for v1perc in np.arange(0.1, 40.0, 0.1):
            v1 = scoreatpercentile(nimage, v1perc)
            if v1 > 0:
                break
        for v2perc in np.arange(99.9, 20.0, -0.1):
            v2 = scoreatpercentile(nimage, v2perc)
            if not np.isnan(v2):
                break
        v1perc_red = v1perc
        v2perc_red = v2perc
        print(v1perc_red, v2perc_red)
        imdisplay(data, x, y, 200, 200, v1perc, v2perc, logscale=True)
        plt.subplots_adjust(wspace=0, hspace=0)
        plt.savefig(datadir + '/png/' + VFID + '-' + galaxy_name + 'red.png', dpi=150)
        plt.close()

    # Append the row to the output_data list
    row = [VFID, galaxy_name, v1perc_blue, v2perc_blue, v1perc_green, v2perc_green, v1perc_red, v2perc_red]
    output_data.append(row)

print('done')

# Write the collected data to a new CSV file
output_csv_file = tabledir + '/galaxy_v1perc_v2perc_data.csv'
with open(output_csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(output_data)
print(f'Data saved to {output_csv_file}')

14.4 40.90000000000336
16.700000000000003 46.90000000000302
21.500000000000004 71.40000000000163
23.000000000000004 80.6000000000011
19.900000000000002 63.500000000002075
21.6 75.3000000000014
18.200000000000003 64.50000000000202
18.6 61.60000000000218
22.1 66.00000000000193
20.1 66.20000000000192
18.1 62.40000000000214
16.700000000000003 61.900000000002166
20.200000000000003 69.90000000000171
11.8 59.30000000000231
17.700000000000003 63.500000000002075
17.8 58.10000000000238
18.1 62.600000000002126
19.900000000000002 61.80000000000217
22.500000000000004 66.6000000000019
19.400000000000002 68.80000000000177
21.500000000000004 68.4000000000018
21.800000000000004 69.20000000000175
20.400000000000002 67.90000000000182
10.9 44.90000000000313
13.200000000000001 42.30000000000328
22.700000000000003 72.70000000000155
22.6 72.30000000000157
22.6 72.30000000000157
21.500000000000004 62.50000000000213
21.700000000000003 74.10000000000147
19.300000000000004 65.30000000000197
19.6 63.8000000000020

In [12]:
#get optical cutouts
csv_file = tabledir+'/Photometrytesting.csv'
galaxy = Table.read(csv_file)
pixscale=1   #standard
pscale = 0
xsize = 0
for i in range(len(galaxy)):
    galaxy_name = str(galaxy['GALAXY'][i])
    path =  datadir+'/pipeline/'+galaxy_name
    destination_folder = path+'/HPPUNIMAPR/'
    partial_name = 'hpacs_25HPPUNIMAPR'
    found_files = find_files(destination_folder, partial_name)
    VFID = f"VFID{int(galaxy['VF_ID'][i]):04d}"
    filename_LS = datadir+'/png/'+VFID+'-'+galaxy_name+'-LS.jpg'
    RA = galaxy['RA_MOMENT'][i]
    DEC = galaxy['DEC_MOMENT'][i]
    if os.path.exists(path):
        if found_files:
            found_file = found_files[0]
            image, head = fits.getdata(found_file, header=True)
            wcs_info = WCS(head) 
            pscale=np.abs(float(head['CDELT1']))   #grab transformation matrix of Herschel image
            xsize=np.abs(int(head['NAXIS1']))   #grab length of Herschel image
            xsize_arcsec=pscale*3600*xsize   #length convert to arcseconds
            imsize=int(xsize_arcsec/pixscale)   #convert length to an integer
            imsize=str(imsize)   #convert integer length to a...string       
            image_url = f'https://www.legacysurvey.org/viewer/cutout.jpg?ra={RA}&dec={DEC}&layer=ls-dr9&size={imsize}&pixscale={1}'
            if os.path.exists(filename_LS):
                os.remove(filename_LS)
                imageLS = wget.download(image_url,out=filename_LS)
            else:
                imageLS = wget.download(image_url,out=filename_LS)
    else:
        print(f"Galaxy not found: {VFID}-{galaxy_name}")
print('done')

100% [............................................................................] 211613 / 211613done


In [9]:
#check which galaxy has images
csv_file = tabledir + '/Photometrytesting.csv'
galaxy = Table.read(csv_file)

# Create new columns if they don't exist
if '70micronsflag' not in galaxy.colnames:
    galaxy['70micronsflag'] = [False] * len(galaxy)

if '100micronsflag' not in galaxy.colnames:
    galaxy['100micronsflag'] = [False] * len(galaxy)

for i in range(len(galaxy)):
    galaxy_name = str(galaxy['GALAXY'][i])
    path = datadir + '/pipeline/' + galaxy_name
    VFID = f"VFID{int(galaxy['VF_ID'][i]):04d}"

    # For 'hpacs_25HPPUNIMAPB'
    destination_folder = path + '\\HPPUNIMAPB'
    partial_name = 'hpacs_25HPPUNIMAPB'
    found_files = find_files(destination_folder, partial_name)
    if found_files:
        galaxy['70micronsflag'][i] = True

    # For 'hpacs_25HPPUNIMAPG'
    destination_folder = path + '\\HPPUNIMAPG'
    partial_name = 'hpacs_25HPPUNIMAPB'
    found_files = find_files(destination_folder, partial_name)
    if found_files:
        galaxy['100micronsflag'][i] = True

# Save the updated table to the CSV file
galaxy.write(csv_file, format='csv', overwrite=True)